In [ ]:
import os
import sys
import numpy as np
import lasagne as L

sys.path.append('../')
sys.path.append('../scripts/')

from os.path import join
from squad_tools import load_dict
from AnswerBot import AnswerBot
from QANet import QANet
from my_tokenize import tokenize

In [35]:
negative = False
train_unk = True

if not negative:
    model_fname = '../models/6B/best/model.ep09.npz'
    conv = 'valid'
else:
    model_fname = '../models/6B/model_neg03/model.ep07.npz'
    conv = 'full'

In [26]:
glove_path = '/pio/data/data/glove_vec/6B/glove'
glove_embs = np.load(join(glove_path, 'glove.6B.300d.npy'))
voc_size = glove_embs.shape[0]

glove_dict = load_dict(join(glove_path, '..', 'glove.6B.300d.txt'))
w_to_i = {w:i for (i,w) in enumerate(glove_dict)}
NAW_token = glove_dict.index('<not_a_word>')

In [36]:
abot = AnswerBot(
    model_fname, glove_embs, glove_dict, '6B',
    train_unk=train_unk,
    negative=negative,
    conv=conv)

Building the model...
Using dropout after wiq calculation.
Compiling theano functions:
    get_intermediate_results_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done


# Demo

In [33]:
x = u"The torch relay started on 24 October 2017 in Greece and ended at "\
     "the start of the Olympics on 9 February 2018. On 1 November 2017 the relay "\
     "entered Korea. The relay lasted 101 days. There were 7,500 torch bearers to "\
     "represent the Korean population of 75 million people. There were also 2018 support "\
     "runners to guard the torch and act as messengers."
    
q1 = u"How long was the torch relay?"
q2 = u"How long were the winter olympics?"
q3 = u"When did the relay end?"
q4 = u"When did the World War II end?"

In [34]:
# negative
abot.get_answers([q1, q2, q3, q4], [x]*4)

[([u'101', u'days'], 0.35858187),
 (['<not_a_word>'], 0.74940485),
 (['<not_a_word>'], 0.26989433),
 (['<not_a_word>'], 0.97529334)]

In [37]:
# positive
abot.get_answers([q1, q2, q3, q4], [x]*4)

[([u'101', u'days'], 0.93867058),
 ([u'101', u'days'], 0.68829244),
 ([u'1', u'november', u'2017'], 0.41395184),
 ([u'9', u'february', u'2018'], 0.49419025)]